In [1]:

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [4]:

# ---- Step 1: Connect to SQLite Database ----
conn = sqlite3.connect('Internship.db')
cursor = conn.cursor()

# ---- Step 2: Queries for Summary ----
cursor.execute("SELECT SUM(Quantity) FROM sales")
total_quantity = cursor.fetchone()[0]

cursor.execute("SELECT SUM(Sales) FROM sales")
total_revenue = cursor.fetchone()[0]

cursor.execute("SELECT AVG(Rating) FROM sales")
avg_rating = cursor.fetchone()[0]

cursor.execute("""
    SELECT [Product line], SUM(Sales) AS total_sales
    FROM sales
    GROUP BY [Product line]
    ORDER BY total_sales DESC
    LIMIT 1
""")
top_product_line, top_sales_value = cursor.fetchone()

# Data for charts
sales_by_product = pd.read_sql_query("""
    SELECT [Product line], SUM(Sales) AS total_sales
    FROM sales
    GROUP BY [Product line]
    ORDER BY total_sales DESC
""", conn)

sales_by_branch = pd.read_sql_query("""
    SELECT Branch, SUM(Sales) AS total_sales
    FROM sales
    GROUP BY Branch
""", conn)

# ---- Step 3: Print Summary ----
print("===== BASIC SALES SUMMARY =====")
print(f"Total Quantity Sold     : {total_quantity}")
print(f"Total Revenue           : {total_revenue:.2f}")
print(f"Average Rating          : {avg_rating:.2f}")
print(f"Top-Selling Product     : {top_product_line} ({top_sales_value:.2f})")
print("\nSales by Branch:")
print(sales_by_branch)

# ---- Step 4: Side-by-Side Charts ----
fig, axes = plt.subplots(1, 2, figsize=(12,5))

# Left chart: Sales by Product Line
bars1 = axes[0].bar(sales_by_product['Product line'], sales_by_product['total_sales'], color='lightblue')
axes[0].set_title("Sales by Product Line")
axes[0].set_xlabel("Product Line")
axes[0].set_ylabel("Total Sales")
axes[0].tick_params(axis='x', rotation=45)

for bar in bars1:
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                 f"{bar.get_height():,.0f}", ha='center', va='bottom', fontsize=8)

# Right chart: Sales by Branch
bars2 = axes[1].bar(sales_by_branch['Branch'], sales_by_branch['total_sales'], color='salmon')
axes[1].set_title("Sales by Branch")
axes[1].set_xlabel("Branch")
axes[1].set_ylabel("Total Sales")

for bar in bars2:
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                 f"{bar.get_height():,.0f}", ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

# Close connection
conn.close()


OperationalError: no such table: sales